In [0]:
!pip install transformers

In [0]:
import tensorflow as tf
import torch

import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


from transformers import BertTokenizer as bertTokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler

from sklearn.svm import SVC
from sklearn import preprocessing

from transformers import BertForSequenceClassification as bfsc,AdamW,BertConfig
from torch.utils.data import Dataset
from transformers import get_linear_schedule_with_warmup



Using TensorFlow backend.


In [0]:
gpuname=""
device=""
y=""
preprocessedTweets=""
ids_of_sentence=[]
ids_of_sentence_words=[]


In [0]:


gpuname=tf.test.gpu_device_name()
if gpuname=='/device:GPU:0':
  print('Found GPU at :{}'.format(gpuname))
else:
  gpuname=""
if torch.cuda.is_available():
  device=torch.device("cuda")
  n_gpu=torch.cuda.device_count()
  print("The device name is %s"%torch.cuda.get_device_name(0))
else:
  print("No GPU available using only CPU instead")
  device=torch.device("cpu")




Found GPU at :/device:GPU:0
The device name is Tesla P100-PCIE-16GB


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
##NOT FOR SYSTEMS
!unzip -P yourpassword -qq '/content/drive/My Drive/EnglishData/task_a_distant.zip' 

replace task_a_distant.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
def convertToFloat(val):
    if not val:
        return 0    
    try:
        return np.float64(val)
    except:        
        return np.float64(0)


headers=['id','text','average','std']
taska = pd.read_csv("task_a_distant.tsv", delimiter='\t',names=headers,low_memory=False,converters={"average":convertToFloat,"std":convertToFloat})
taska=taska[1:]

In [0]:
taska.dtypes

id          object
text        object
average    float64
std        float64
dtype: object

In [0]:
len(taska)
#taska=taska[:1000000]
#taska=taska[100000:4000000]
#taska=taska[4000000:6000000]
#taska=taska[6000000:]
#taska=taska[:1000000]
#taska=taska[1000000:1500000]
#taska=taska[1500000:2500000]
#taska=taska[3500000:4500000]
taska=taska[6000000:7500000]


In [0]:

#GET THE DATA FROM THE PANDAS FRAME
headers=['id','tweet','subtask_a','subtask_b','subtask_c']
englishdata = pd.read_csv("olid-training-v1.0.tsv", delimiter='\t',names=headers,low_memory=False)
englishdata=englishdata[['id','tweet','subtask_a']]
englishdata=englishdata[1:]

In [0]:
englishdata.head()

,id,tweet,subtask_a
1,86426,@USER She should ask a few native Americans wh...,OFF
2,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
3,16820,Amazon is investigating Chinese employees who ...,NOT
4,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF
5,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT


In [0]:
ids_of_sentence=[]
ids_of_sentence_words=[]
attention_masks=[]

def giveIds(sentence):
  maxlength=0
  tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
  for t in sentence:
      tokenized_sentence_id=tokenizer.encode(t,add_special_tokens=True)
      if(maxlength<len(tokenized_sentence_id)):
          maxlength=len(tokenized_sentence_id)
  return maxlength

In [0]:
dfnumpy=englishdata.to_numpy()
x_train=dfnumpy[:, 1].reshape(-1, 1)
y_train=dfnumpy[:, 2].reshape(-1, 1)

In [0]:
type(y_train)

numpy.ndarray

In [0]:

def giveLabel(y):
  i=0
  y1=[]
  for r in y:
    if(y[i]>=0.5):
      y1.append(1)
    else:
      y1.append(0)
    i=i+1;
  return y1

headers=['id','text','average','std']

dfnumpy=taska.to_numpy();
x_train=dfnumpy[:, 1].reshape(-1, 1)
y_train=dfnumpy[:, 2].reshape(-1, 1)
y1=giveLabel(y_train)

In [0]:
dfnumpy=englishdata.to_numpy()
x_test=dfnumpy[:, 1].reshape(-1, 1)
y_test=dfnumpy[:, 2].reshape(-1, 1)

In [0]:
from sklearn.utils import shuffle
x_train,y_train=shuffle(x_train,y1)

In [0]:
x_train

array([['most of the time when people wish yg to get bankrupt i cant help but feel anxious and worried for treasure13'],
       ['I need some new everyday sunglasses'],
       ['It ain’t enough for me to win I gotta win with my team man'],
       ...,
       ['Richard Madeley is one of those people who, when you see them trending you know it’s just because they’re being a knob as usual.'],
       ["@USER You support Trump's hate speech"],
       ['The veracity that this tweet holds 💯no matter what happen, my loyalty is still with the montefalcos']],
      dtype=object)

In [0]:
y_test

array([['OFF'],
       ['OFF'],
       ['NOT'],
       ...,
       ['OFF'],
       ['OFF'],
       ['NOT']], dtype=object)

In [0]:
y_test

array([['OFF'],
       ['OFF'],
       ['NOT'],
       ...,
       ['OFF'],
       ['OFF'],
       ['NOT']], dtype=object)

In [0]:
from sklearn.svm import SVC
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

#print(y_train.shape)
print(y_test.shape)
#print(y_predict.shape)
yTrain=le.fit_transform(y_train)
print(yTrain.shape)
print(le.classes_)
yTest=le.fit_transform(y_test.flatten())
print(le.classes_)

(13240, 1)
(1000000,)
[0 1]
['NOT' 'OFF']


In [0]:
set(yTrain)

{0, 1}

In [0]:
from transformers import BertForSequenceClassification as bfsc,AdamW,BertConfig
model=bfsc.from_pretrained('bert-base-uncased',num_labels=2,output_attentions=False,output_hidden_states=False)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
##ONLY FOR THE FIRST TIME SAVE ELSE LOAD
torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/EnglishData/bertenglish1.pth.tar')
checkpoint = torch.load('/content/drive/My Drive/EnglishData/bertenglish1.pth.tar')
model.load_state_dict(checkpoint['state_dict'])
  

<All keys matched successfully>

In [0]:
params=list(model.named_parameters())

In [0]:
no_decay = ["bias", "beta","LayerNorm.weight","gamma"]
optimizer_grouped_parameters = [
{
"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
"weight_decay": 0.01,
},
{"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]

In [0]:
optimizer=AdamW(model.parameters(),lr=2e-5,eps=1e-8)

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def calculateF1Score(predictions,labels):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=np.argmax(predictions,axis=1).flatten()
  labelsFlattend=labels.flatten()
  print("Predictions Argmax",predictionArgmax)
  print("labels Flattened",labelsFlattend)   
  return f1_score(labelsFlattend, predictionArgmax, average='macro'),accuracy_score(labelsFlattend, predictionArgmax)



In [0]:
MAXLENGTH=64

In [0]:
class EnglishTrainDataset(Dataset):
    def __init__(self,xytrain):
        self.xytrain = xytrain
        self.maxlength=MAXLENGTH
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytrain[0][index].flatten()))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_train_pytorch = torch.tensor(ids_of_sentence_word)
        y_train_pytorch=torch.tensor(self.xytrain[1][index])
        x_train_mask_pytorch=torch.tensor(attention_mask)
        
        return x_train_pytorch,x_train_mask_pytorch,y_train_pytorch
        #return [1,2,3]
        
    def __len__(self):
        return len(self.xytrain[0])
 


'\nz=0;\nfor batch_idx, data in enumerate(tdataloader): \n  if z==100:\n    break;\n  z=z+1;'

In [0]:
class EnglishTestDataset(Dataset):
    def __init__(self,xytest):
        self.xytest = xytest
        self.maxlength=MAXLENGTH
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytest[0][index].flatten()))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_test_pytorch = torch.tensor(ids_of_sentence_word)
        y_test_pytorch=torch.tensor(self.xytest[1][index])
        x_test_mask_pytorch=torch.tensor(attention_mask)
        
        return x_test_pytorch,x_test_mask_pytorch,y_test_pytorch
        #return [1,2,3]
        
    def __len__(self):
        return len(self.xytest[0])



In [0]:
#xytrain=[x_train[:8000],y_train[:8000]]\
MAXLENGTH=64
xytrain=[x_train,yTrain]
tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
tdataset = EnglishTrainDataset(xytrain)
tsampler=RandomSampler(tdataset)
tdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)
      



In [0]:

#xytest=[x_test[:8000],y_test[:8000]]
xytest=[x_test,yTest]
tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
tedataset = EnglishTestDataset(xytest) 
tesampler=RandomSampler(tedataset)
tedataloader = DataLoader(tedataset, batch_size=32, num_workers=1, shuffle=False,sampler=tesampler)
#trainData(tdataloader,tedataloader)

In [0]:
epochs=2
total_steps=len(tdataloader)*epochs
sch=get_linear_schedule_with_warmup(optimizer,
                                    num_warmup_steps=0,num_training_steps=total_steps)

In [0]:
import random
import time 

def set_seed(seed,ngpu):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if ngpu > 0:
        torch.cuda.manual_seed_all(seed)
      
set_seed(42,torch.cuda.device_count())
#remove later

epochs=2
lossList=[]
max_grad_norm=1.0
for e in range(0, epochs):
    
    print("Start Epoch Number",(e + 1))
    print("Start Training")
    if device.type=="cpu":
     model.to(device)
     map_location='cpu'
    else:
      model.cuda()
      map_location=lambda storage, loc: storage.cuda()
    checkpoint = torch.load('/content/drive/My Drive/EnglishData/bertenglish1.pth.tar',map_location=map_location)
    model.load_state_dict(checkpoint['state_dict'])

    #Amount of time taken for training
    t1 = time.time()
    tr_loss, logging_loss = 0.0, 0.0
    model.train()
    tsteps=0
    for step, batch in enumerate(tdataloader):
        if step % 50 == 0 and not step == 0:
            print("Batch Completed  {:,}  of  {:,}.    Elapsed time is  {}".format(step, len(tdataloader),time.time() - t1))
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        model.zero_grad()        
        outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"], labels=inputs["labels"])
        loss = outputs[0]
        loss.backward()
        tr_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        tsteps+=1
        optimizer.step()
        sch.step()
    a_tr_loss = tr_loss /(tsteps)               
    lossList.append(a_tr_loss)
    print(" The training loss incured is  {0:.3f}".format(a_tr_loss))
    t2=time.time()
    print("  Training one epoch time taken",t2-t1)
    print(" Validation starts here ")
    t1 = time.time()
    model.eval()
    eval_loss = 0
    nb_eval_steps = 0
    eval_f1=0
    eval_acc=0
    
    for batch_idx, data in enumerate(tedataloader):
        
        batch = tuple(t.to(device) for t in data)            
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        with torch.no_grad():        
           outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"])
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = (inputs["labels"]).to('cpu').numpy()
        tmpf1score,tmpaccscore = calculateF1Score(logits, label_ids)
        eval_f1 = eval_f1+tmpf1score
        eval_acc=eval_acc+tmpaccscore
        nb_eval_steps += 1
        #print(" TEMP F1 score: {0:.3f}".format(tmpf1score))
        #print("TEMP  Accuracy score: {0:.3f}".format(tmpaccscore))
    torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/EnglishData/bertenglish1.pth.tar')
    print("  F1 score: {0:.3f}".format(eval_f1/nb_eval_steps))
    print("  Accuracy score: {0:.3f}".format(eval_acc/nb_eval_steps))
    t2=time.time()
    print("  Validating one epoch time taken ",t2-t1)
      
print("ALL DONE!!!")

Start Epoch Number 1
Start Training
Batch Completed  50  of  31,250.    Elapsed time is  10.933725357055664
Batch Completed  100  of  31,250.    Elapsed time is  21.56212043762207
Batch Completed  150  of  31,250.    Elapsed time is  32.175060510635376
Batch Completed  200  of  31,250.    Elapsed time is  42.78130006790161
Batch Completed  250  of  31,250.    Elapsed time is  53.36810874938965
Batch Completed  300  of  31,250.    Elapsed time is  63.989893674850464
Batch Completed  350  of  31,250.    Elapsed time is  74.59470796585083
Batch Completed  400  of  31,250.    Elapsed time is  85.23959922790527
Batch Completed  450  of  31,250.    Elapsed time is  95.87529182434082
Batch Completed  500  of  31,250.    Elapsed time is  106.5081799030304
Batch Completed  550  of  31,250.    Elapsed time is  117.13708019256592
Batch Completed  600  of  31,250.    Elapsed time is  127.79792261123657
Batch Completed  650  of  31,250.    Elapsed time is  138.39516162872314
Batch Completed  700  o